# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(ws, name="Hyperdrive_run")

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: feature-test
Azure region: germanywestcentral
Subscription id: 779ddc6a-162c-43db-9f05-edaaa1cde3ba
Resource group: rg-preview-features


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

#Create compute cluster
aml_compute_name="cpu-compute"

#verify that cluster does not already exist
try:
    aml_compute=ComputeTarget(workspace=ws, name=aml_compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_DS2_V2",max_nodes=5)
    aml_compute=ComputeTarget.create(ws,aml_compute_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C" : choice(0.01, 0.1, 1),
    "--max_iter" : choice(20, 40, 60, 100, 150, 200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yaml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target="cpu-compute",
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=ps,
                    policy=policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=4,
                    max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_62c536e8-f828-4544-908b-8d80dae75070
Web View: https://ml.azure.com/runs/HD_62c536e8-f828-4544-908b-8d80dae75070?wsid=/subscriptions/779ddc6a-162c-43db-9f05-edaaa1cde3ba/resourcegroups/rg-preview-features/workspaces/feature-test&tid=ce849bab-cc1c-465b-b62e-18f07c9ac198

Streaming azureml-logs/hyperdrive.txt

[2023-04-23T10:21:34.450201][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-04-23T10:21:34.8753119Z][SCHEDULER][INFO]Scheduling job, id='HD_62c536e8-f828-4544-908b-8d80dae75070_0' 
[2023-04-23T10:21:35.0193061Z][SCHEDULER][INFO]Scheduling job, id='HD_62c536e8-f828-4544-908b-8d80dae75070_1' 
[2023-04-23T10:21:35.1170532Z][SCHEDULER][INFO]Scheduling job, id='HD_62c536e8-f828-4544-908b-8d80dae75070_2' 
[2023-04-23T10:21:35.195195][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-04-23T10:21:35.2379897Z][SCHEDULER][INFO]Scheduling job, id='HD_62c536e8-f828-4544-908b-8d80dae75070_3' 
[2023-

{'runId': 'HD_62c536e8-f828-4544-908b-8d80dae75070',
 'target': 'cpu-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-04-23T10:21:33.673263Z',
 'endTimeUtc': '2023-04-23T10:25:07.155341Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '33be1182-e2cc-4efc-bfb6-1fdfae473478',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': '18',
  'score': '0.7676767676767676',
  'best_child_run_id': 'HD_62c536e8-f828-4544-908b-8d80dae75070_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_62c536e8-f828-4544-908b-8d80dae75070_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClient

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [6]:
import joblib
# Get your best run and save the model from that run.
best_run=hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
parameter_values=best_run.get_details()['runDefinition']['arguments']

print('Best Run ID:',best_run.id)
print('\n Accuracy:',best_run_metrics['Accuracy'])

Best Run ID: HD_62c536e8-f828-4544-908b-8d80dae75070_1

 Accuracy: 0.7676767676767676


In [7]:
print(best_run)
best_run

Run(Experiment: Hyperdrive_run,
Id: HD_62c536e8-f828-4544-908b-8d80dae75070_1,
Type: azureml.scriptrun,
Status: Completed)


Experiment,Id,Type,Status,Details Page,Docs Page
Hyperdrive_run,HD_62c536e8-f828-4544-908b-8d80dae75070_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [8]:
model=best_run.register_model(model_name='hyperdrive_model',model_path='./outputs/model.joblib')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

